In [7]:
import pandas as pd 

In [12]:
rankings = pd.read_csv('ticker_ranking.csv')
tickers = pd.read_csv('tickers.csv')

In [16]:
tickers.head()

,Symbol,Name
0,42,42 Coin
1,300,300 token
2,365,365Coin
3,404,404Coin
4,433,433 Token


In [17]:
from fuzzywuzzy import fuzz
import operator
import itertools


In [23]:
score_threshold = 85

combinations = list(itertools.product(rankings['Currency Name'].values.ravel(), tickers['Name'].values.ravel()))
scores = {x: fuzz.ratio(x[0], x[1]) for x in combinations}
matches = [max({k:v for k,v in scores.items() if k[0] == x}.items(), key=operator.itemgetter(1)) for x in rankings['Currency Name'].values.ravel()]
matches = {x[0][0]:x[0][1] for x in matches if x[1] >= score_threshold}

In [29]:
not_found = list(set(rankings['Currency Name'].values.ravel()) - set(matches.keys()))
not_found = {x:x for x in not_found}

matches = dict(**matches, **not_found)

In [30]:
rankings['Currency Name'] = rankings['Currency Name'].apply(lambda x: matches[x])

In [36]:
ticker_dict = {x[0]:x[1] for x in tickers[['Name', 'Symbol']].values}

In [49]:
rankings['Ticker'] = rankings['Currency Name'].apply(lambda x: ticker_dict[x] if x in ticker_dict.keys() else "NOT_FOUND")

In [51]:
rankings.to_csv('ticker_ranking.csv', index = False)